In [ ]:
import datasets
import tiktoken

In [2]:
data = datasets.load_dataset("liaad/PTradutor")

Generating test split: 100%|██████████| 1842/1842 [00:00<00:00, 187185.83 examples/s]


#### Count the number of tokens

In [3]:
encoder = tiktoken.get_encoding("cl100k_base")

On the train set.

In [4]:
n_tokens_train_en = sum(len(encoder.encode(example["en"])) for example in data["train"])
print(f"Number of tokens in the training set in english: {n_tokens_train_en}")

Number of tokens in the training set in english: 322256277


In [5]:
n_tokens_train_pt = sum(len(encoder.encode(example["pt"])) for example in data["train"])
print(f"Number of tokens in the training set in portuguese: {n_tokens_train_pt}")

Number of tokens in the training set in portuguese: 424131611


In [6]:
print(f"Number of tokens in the training set in total: {n_tokens_train_en + n_tokens_train_pt}")

Number of tokens in the training set in total: 746387888


On the test set.

In [7]:
n_tokens_test_en = sum(len(encoder.encode(example["en"])) for example in data["test"])
print(f"Number of tokens in the test set in english: {n_tokens_test_en}")

Number of tokens in the test set in english: 74969


In [8]:
n_tokens_test_pt = sum(len(encoder.encode(example["pt"])) for example in data["test"])
print(f"Number of tokens in the test set in portuguese: {n_tokens_test_pt}")

Number of tokens in the test set in portuguese: 101001
